19th

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Fully Connected Network (FCN)
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convolutional Neural Network (CNN) - Fixed Input Size
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Downsample to (64, 14, 14)
        
        self.fc1 = nn.Linear(64 * 14 * 14, 128)  # Fixed input size
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))  # Output shape (64, 14, 14)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to train and evaluate models
def train_and_evaluate(model, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    start_time = time.time()
    
    # Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")
    
    training_time = time.time() - start_time

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    num_params = sum(p.numel() for p in model.parameters())
    
    return num_params, training_time, accuracy

# Run comparison
print("\nTraining Fully Connected Network (FCN)...")
fcn_model = FCN()
fcn_params, fcn_time, fcn_acc = train_and_evaluate(fcn_model)

print("\nTraining Convolutional Neural Network (CNN)...")
cnn_model = CNN()
cnn_params, cnn_time, cnn_acc = train_and_evaluate(cnn_model)

# Print results
print("\n===== Model Performance Summary =====")
print(f"Fully Connected Network (FCN): Parameters={fcn_params}, Training Time={fcn_time:.2f}s, Test Accuracy={fcn_acc:.2f}%")
print(f"Convolutional Neural Network (CNN): Parameters={cnn_params}, Training Time={cnn_time:.2f}s, Test Accuracy={cnn_acc:.2f}%")



Training Fully Connected Network (FCN)...
Epoch [1/5], Loss: 0.4986
Epoch [2/5], Loss: 0.3618
Epoch [3/5], Loss: 0.3244
Epoch [4/5], Loss: 0.3011
Epoch [5/5], Loss: 0.2809

Training Convolutional Neural Network (CNN)...
Epoch [1/5], Loss: 0.3921
Epoch [2/5], Loss: 0.2405
Epoch [3/5], Loss: 0.1906
Epoch [4/5], Loss: 0.1555
Epoch [5/5], Loss: 0.1226

===== Model Performance Summary =====
Fully Connected Network (FCN): Parameters=535818, Training Time=279.92s, Test Accuracy=87.91%
Convolutional Neural Network (CNN): Parameters=1625866, Training Time=906.20s, Test Accuracy=92.26%
